In [1]:
import numpy as np
import pandas as pd

from load import load_staph

pd.options.display.precision = 3
pd.options.display.max_colwidth = 12

%matplotlib inline

In [2]:
records = load_staph(True)
mask = records['resp'].notna()
records.head()

,id,sequence,missing,missing_%,sequence_i,missing_i,missing_%_i,resp,Total.Area
0,NRS001,ATGAACAT...,2511,0.255,ATGAACAT...,2356,0.240,False,0.000
1,NRS002,--------...,25278,2.571,ATGAACAT...,2236,0.227,False,0.000
2,NRS003,ATGAACAT...,48213,4.904,ATGAACAT...,2253,0.229,False,0.000
3,NRS021,ATGAAAAT...,2442,0.248,ATGAAAAT...,2088,0.212,False,473.152
4,NRS022,ATGAACAT...,3885,0.395,ATGAACAT...,2154,0.219,False,6686.806


In [3]:
import os

random_state = 42

s = {'{}_{}_{}_{}.npy'.format(impute, nc, selection, extraction)
     for impute in 'io'
     for nc in 'nc'
     for selection in '-vx'
     for extraction in '-pts'}

# numerical data
data_u = {d: np.load(os.path.join('../data/staph/preprocess', d)) for d in s}
for k, v in data_u.items():
    if v.shape[0] != 124:
        print(k)

# one-hot encoded data
data_e = {d: np.load(os.path.join('../data/staph/preprocess/onehot', d)) for d in s}

In [4]:
from sklearn.model_selection import train_test_split

# Classification

In [5]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [6]:
# silence warning for getting 0 on f1 score
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

In [7]:
y_c = records['resp'][mask].astype('?')
metrics_c = [accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score]

In [8]:
def train_clfs(clf, model, encode):
    results = []
    data = data_e if encode else data_u
    for d, X in data.items():
        X_train, X_test, y_train, y_test = train_test_split(X, y_c, random_state=random_state,
                                                            stratify=y_c, train_size=0.7)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        m = {metric.__name__.replace('_score', ''): metric(y_test, y_pred)
             for metric in metrics_c}
        m.update({'file': d, 'encode': encode, 'model': model})
        results.append(m)
    return results

### Logistic regression

In [9]:
# 4*2 minutes
lr = LogisticRegression(penalty='none', class_weight='balanced',
                         solver='lbfgs', max_iter=2000, n_jobs=5, warm_start=False)
%time lr_u = train_clfs(lr, 'logistic', False)
%time lr_e = train_clfs(lr, 'logistic', True)

CPU times: user 1min 37s, sys: 8.6 s, total: 1min 45s
Wall time: 3min 51s
CPU times: user 4min 21s, sys: 14.5 s, total: 4min 35s
Wall time: 4min 28s


###  Random forest

In [10]:
# 1*2 minutes
rfc = RandomForestClassifier(n_estimators=500, n_jobs=5, class_weight='balanced')
%time rfc_u = train_clfs(rfc, 'rf', False)
%time rfc_e = train_clfs(rfc, 'rf', True)

CPU times: user 1min 13s, sys: 9.98 s, total: 1min 23s
Wall time: 54.6 s
CPU times: user 1min 11s, sys: 7.85 s, total: 1min 18s
Wall time: 47.5 s


###  Support vector machine

In [11]:
# 1.75 and 2.5 minutes
svc = SVC(gamma='auto', class_weight='balanced')
%time svc_u = train_clfs(svc, 'svm', False)
%time svc_e = train_clfs(svc, 'svm', True)

CPU times: user 1min 31s, sys: 3.11 s, total: 1min 35s
Wall time: 1min 35s
CPU times: user 2min 13s, sys: 2.34 s, total: 2min 15s
Wall time: 2min 16s


### Results

In [12]:
results_c = pd.DataFrame(lr_e + lr_u + rfc_u + rfc_e + svc_u + svc_e)
results_c.to_csv('result/result_staph_clf.csv', index=False)
results_c.head()

,accuracy,balanced_accuracy,precision,recall,f1,file,encode,model
0,0.842,0.50,0.0,0.000,0.000,i_c_-_t.npy,True,logistic
1,0.842,0.50,0.0,0.000,0.000,i_n_-_p.npy,True,logistic
2,0.842,0.50,0.0,0.000,0.000,o_n_-_t.npy,True,logistic
3,0.816,0.62,0.4,0.333,0.364,i_c_-_-.npy,True,logistic
4,0.842,0.50,0.0,0.000,0.000,i_c_-_s.npy,True,logistic


In [13]:
results_c.sort_values(by=['f1'], ascending=False)

,accuracy,balanced_accuracy,precision,recall,f1,file,encode,model
48,0.711,0.828,0.353,1.000,0.522,i_c_-_t.npy,False,logistic
113,0.895,0.667,1.000,0.333,0.500,i_n_-_s.npy,False,rf
59,0.895,0.667,1.000,0.333,0.500,o_n_v_-.npy,False,logistic
155,0.895,0.667,1.000,0.333,0.500,o_n_v_-.npy,True,rf
218,0.895,0.667,1.000,0.333,0.500,o_n_-_-.npy,False,svm
...,...,...,...,...,...,...,...,...
146,0.842,0.500,0.000,0.000,0.000,o_n_-_t.npy,True,rf
148,0.842,0.500,0.000,0.000,0.000,i_c_-_s.npy,True,rf
149,0.842,0.500,0.000,0.000,0.000,i_n_v_t.npy,True,rf
151,0.842,0.500,0.000,0.000,0.000,i_c_v_p.npy,True,rf


# Regression

In [6]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [7]:
y_r = records['resp'][mask]
metrics_r = [r2_score, mean_squared_error]

In [8]:
def train_regs(reg, model, encode):
    results = []
    data = data_e if encode else data_u
    for d, X in data.items():
        X_train, X_test, y_train, y_test = train_test_split(X, y_r, random_state=random_state,
                                                            train_size=0.7)
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        m = {metric.__name__.replace('_score', ''): metric(y_test, y_pred)
             for metric in metrics_r}
        m.update({'file': d, 'encode': encode, 'model': model})
        results.append(m)
    return results

### Linear regression

In [9]:
# 30*2 seconds
lrr = LinearRegression(n_jobs=5)
%time lrr_u = train_regs(lrr, 'linear', False)
%time lrr_e = train_regs(lrr, 'linear', True)

CPU times: user 8min 47s, sys: 26 s, total: 9min 13s
Wall time: 31.5 s
CPU times: user 11min 53s, sys: 25.1 s, total: 12min 18s
Wall time: 29.7 s


### Random forest

In [10]:
# 13 and 25 minutes
rfr = RandomForestRegressor(n_estimators=500, n_jobs=5)
%time rfr_u = train_regs(rfr, 'rf', False)
%time rfr_e = train_regs(rfr, 'rf', True)

CPU times: user 1h 5min 12s, sys: 8.3 s, total: 1h 5min 20s
Wall time: 13min 37s
CPU times: user 2h 3min 31s, sys: 7.35 s, total: 2h 3min 38s
Wall time: 25min 14s


### Support vector machine

In [11]:
# 1.5*2 minutes
svr = SVR(gamma='auto')
%time svr_u = train_regs(svr, 'svm', False)
%time svr_e = train_regs(svr, 'svm', True)

CPU times: user 1min 10s, sys: 2.57 s, total: 1min 13s
Wall time: 1min 13s
CPU times: user 1min 26s, sys: 1.95 s, total: 1min 28s
Wall time: 1min 28s


In [12]:
results_r = pd.DataFrame(lrr_e + lrr_u + rfr_u + rfr_e + svr_u + svr_e)
results_r.to_csv('result/result_staph_reg.csv', index=False)
results_r.head()

,r2,mean_squared_error,file,encode,model
0,-0.040,0.188,i_c_-_s.npy,True,linear
1,-0.382,0.250,o_c_x_-.npy,True,linear
2,-0.054,0.191,o_n_x_s.npy,True,linear
3,-0.052,0.190,i_n_-_p.npy,True,linear
4,-0.038,0.188,i_c_v_t.npy,True,linear


In [15]:
results_r.sort_values(by=['mean_squared_error'])

,r2,mean_squared_error,file,encode,model
127,0.198,0.145,i_c_-_t.npy,False,rf
233,0.182,0.148,i_c_-_-.npy,False,svm
139,0.108,0.161,o_n_v_p.npy,False,rf
123,0.104,0.162,i_c_v_p.npy,False,rf
216,0.101,0.162,i_n_x_-.npy,False,svm
...,...,...,...,...,...
76,-37.295,6.922,o_n_-_s.npy,False,linear
66,-48.700,8.983,i_c_x_s.npy,False,linear
84,-74.312,13.612,i_n_-_s.npy,False,linear
50,-77.714,14.227,o_n_x_s.npy,False,linear
